In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/supply-chain-analysis/supply_chain_data.csv


In [2]:
import pandas as pd
import duckdb
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('/kaggle/input/supply-chain-analysis/supply_chain_data.csv')

In [4]:
print("\n--- First 5 rows of my dataset ---")
df.head()


--- First 5 rows of my dataset ---


,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [5]:
print("\n--- Columns in my dataset ---")
print(df.columns)


--- Columns in my dataset ---
Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')


In [6]:
print("\n--- Dataset Information ---")
print(df.info())


--- Dataset Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14

In [7]:
print("\n--- Missing values per column ---")
print(df.isnull().sum())


--- Missing values per column ---
Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64


In [8]:
print("\n--- Number of duplicate rows ---")
print(df.duplicated().sum())


--- Number of duplicate rows ---
0


### Total Revenue Generated

In [9]:
# Calculating the total revenue using a SQL query with duckdb.
query_total_revenue = """
    SELECT SUM("Revenue generated")::DECIMAL(8, 2) AS total_revenue
    FROM df
"""
total_revenue_df = duckdb.query(query_total_revenue).df()

my_total_revenue = total_revenue_df["total_revenue"][0]

print(f"\nMy calculated total revenue: ${my_total_revenue:,.2f}")


My calculated total revenue: $577,604.82


In [10]:
fig_total_revenue = go.Figure()
fig_total_revenue.add_trace(go.Indicator(
    mode = "number",
    value = my_total_revenue,
    title = {"text": "Total Revenue Generated"},
    number = {"prefix": "$", "valueformat": ".2f"},
    domain = {"x": [0, 1], "y": [0, 1]}
))

In [11]:
fig_total_revenue.update_layout(
    font=dict(size=18),
    font_color = "white",
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

fig_total_revenue.show()

### Revenue by Product Type


In [12]:
query_revenue_by_product = """
    SELECT "Product Type",
           SUM("Revenue generated")::DECIMAL(8, 2) AS total_revenue
    FROM df
    GROUP BY "Product Type"
    ORDER BY total_revenue DESC
"""
revenue_by_product_df = duckdb.query(query_revenue_by_product).df()

print("\n--- Revenue by Product Type ---")
print(revenue_by_product_df)


--- Revenue by Product Type ---
  Product type  total_revenue
0     skincare      241628.16
1     haircare      174455.39
2    cosmetics      161521.27


In [13]:
fig_revenue_by_product = px.bar(revenue_by_product_df, 
             x='Product type', 
             y='total_revenue', 
             title='Revenue Generated by Product Type',
             labels={'total_revenue': 'Total Revenue ($)'})

fig_revenue_by_product.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Total Revenue ($)",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    margin=dict(l=40, r=40, t=40, b=40),
    font=dict(size=14),
    font_color='white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    bargap=0,
    bargroupgap=0.1
)

fig_revenue_by_product.update_traces(marker=dict(color=['#813cf6', '#15abbd', '#df9def']))
fig_revenue_by_product.show()

### Revenue Distribution by Location


In [14]:
query_revenue_by_location = """
    SELECT "Location",
           SUM("Revenue generated")::DECIMAL(8, 2) AS total_revenue
    FROM df
    GROUP BY "Location"
    ORDER BY total_revenue DESC
"""
revenue_by_location_df = duckdb.query(query_revenue_by_location).df()

print("\n--- Revenue by Location ---")
print(revenue_by_location_df)


--- Revenue by Location ---
    Location  total_revenue
0     Mumbai      137755.03
1    Kolkata      137077.55
2    Chennai      119142.82
3  Bangalore      102601.72
4      Delhi       81027.70


In [15]:
# Pie chart for revenue distribution by location
fig_revenue_by_location = px.pie(
    revenue_by_location_df, 
    values='total_revenue', 
    names='Location', 
    title='Revenue Distribution by Location',
    labels={'total_revenue': 'Total Revenue ($)'},
    hover_name='Location',
    hover_data={'total_revenue': ':.2f'}
)

fig_revenue_by_location.update_layout(
    margin=dict(l=40, r=40, t=40, b=40),
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
    legend=dict(
        title='Location',
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=0
    )
)

fig_revenue_by_location.update_traces(
    marker=dict(colors=['#d62728', '#e377c2', '#ff7f0e', '#ffbb78', '#ff9896'])
)

fig_revenue_by_location.show()

### Percentage Contribution of Revenue by Location


In [16]:
# Calculating the percentage contribution of revenue by location
query_percentage_revenue_by_location = """
    SELECT "Location",
           SUM("Revenue generated") AS total_revenue
    FROM df
    GROUP BY "Location"
    ORDER BY total_revenue DESC
"""

# Executing the query using DuckDB and converting the result to a DataFrame
percentage_revenue_by_location_df = duckdb.query(query_percentage_revenue_by_location).df()

# Calculating percentage contribution of each location
percentage_revenue_by_location_df["percentage_contribution"] = (
    percentage_revenue_by_location_df["total_revenue"] /
    percentage_revenue_by_location_df["total_revenue"].sum()
) * 100

# Rounding the percentage values to 2 decimal places
percentage_revenue_by_location_df["percentage_contribution"] = \
    percentage_revenue_by_location_df["percentage_contribution"].round(2)

# Sorting the DataFrame by total revenue in descending order
percentage_revenue_by_location_df = percentage_revenue_by_location_df.sort_values(
    by='total_revenue', ascending=False
)

# Displaying the result
print("\n--- Percentage Contribution of Revenue by Location ---")
print(percentage_revenue_by_location_df)


--- Percentage Contribution of Revenue by Location ---
    Location  total_revenue  percentage_contribution
0     Mumbai  137755.026877                    23.85
1    Kolkata  137077.551005                    23.73
2    Chennai  119142.815748                    20.63
3  Bangalore  102601.723882                    17.76
4      Delhi   81027.701225                    14.03


In [17]:
# Bar chart for percentage contribution of revenue by location
fig_percentage_revenue_by_location = px.bar(
    percentage_revenue_by_location_df, 
    x='Location', 
    y='percentage_contribution', 
    title='Percentage Contribution of Revenue by Location',
    labels={'percentage_contribution': 'Percentage of Total Revenue (%)'}
)

# Customizing the layout
fig_percentage_revenue_by_location.update_layout(
    xaxis_title="Location",
    yaxis_title="Percentage of Total Revenue (%)",
    yaxis_tickformat=".2f",
    margin=dict(l=40, r=40, t=40, b=40),
    font=dict(size=14),
    font_color='white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',  
    bargap=0.2,
    bargroupgap=0.1
)

# Setting custom bar colors
fig_percentage_revenue_by_location.update_traces(
    marker=dict(color=['#8808f3', '#cc1b1a', '#265f01', '#6ee377', '#4dc640'])
)

# Displaying the chart
fig_percentage_revenue_by_location.show()

## Inventory and Order Management


### Total Stock Levels and Lead Times


In [18]:
# Summing up stock levels and lead times.
query_stock_lead_times = """
    SELECT 
        SUM("Stock levels") AS "Total Stock Levels",
        SUM("Lead times") AS "Total Lead Times"
    FROM 
        df;
"""
stock_lead_times_df = duckdb.query(query_stock_lead_times).df()

my_total_stock_levels = stock_lead_times_df["Total Stock Levels"][0]
my_total_lead_times = stock_lead_times_df["Total Lead Times"][0]

print(f"\nMy total stock levels: {my_total_stock_levels}")
print(f"My total lead times: {my_total_lead_times}")


My total stock levels: 4777.0
My total lead times: 1596.0


In [19]:
# Visualizing total stock levels and lead times with gauge charts

# Gauge chart for Total Stock Levels
fig_stock_levels = go.Figure(go.Indicator(
    mode="number+gauge",
    value=my_total_stock_levels,
    gauge={
        'axis': {'range': [0, max(my_total_stock_levels, my_total_lead_times) + 100]},
        'bar': {'color': "rgba(31, 119, 180, 0.8)"},
        'steps': [
            {'range': [0, max(my_total_stock_levels, my_total_lead_times) / 2], 'color': "lightgray"},
            {'range': [max(my_total_stock_levels, my_total_lead_times) / 2, max(my_total_stock_levels, my_total_lead_times)], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': my_total_stock_levels
        }
    }
))
fig_stock_levels.update_layout(
    title={'text': "Total Stock Levels", 'font': {'size': 20}},
    font=dict(size=18, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Gauge chart for Total Lead Times
fig_lead_times = go.Figure(go.Indicator(
    mode="number+gauge",
    value=my_total_lead_times,
    gauge={
        'axis': {'range': [0, max(my_total_stock_levels, my_total_lead_times) + 100]},
        'bar': {'color': "rgba(214, 39, 40, 0.8)"},
        'steps': [
            {'range': [0, max(my_total_stock_levels, my_total_lead_times) / 2], 'color': "lightgray"},
            {'range': [max(my_total_stock_levels, my_total_lead_times) / 2, max(my_total_stock_levels, my_total_lead_times)], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': my_total_lead_times
        }
    }
))
fig_lead_times.update_layout(
    title={'text': "Total Lead Times", 'font': {'size': 20}},
    font=dict(size=18, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display both gauge charts
fig_stock_levels.show()
fig_lead_times.show()

### Total Order Quantities


In [20]:
# Calculating total order quantities.
query_total_orders = """
    SELECT 
        SUM("Order quantities") AS "Total Orders Quantity"
    FROM 
        df;
"""
total_orders_result = duckdb.query(query_total_orders).fetchall()

my_total_orders_quantity = total_orders_result[0][0]

print(f"\nMy total quantity of orders placed: {my_total_orders_quantity}")


My total quantity of orders placed: 4922


In [21]:
# Displaying the total order quantity as a numeric indicator
import plotly.graph_objects as go

fig_total_orders = go.Figure()

fig_total_orders.add_trace(go.Indicator(
    mode="number",
    value=my_total_orders_quantity,
    title={"text": "Total Orders Quantity"},
    number={"valueformat": ",.0f"}  # Formats the number with commas and no decimals
))

fig_total_orders.update_layout(
    font=dict(size=18, color='black'),
    margin=dict(l=20, r=20, t=80, b=20),
    paper_bgcolor='white',  # Solid white background
    plot_bgcolor='white'
)

# Display the chart
fig_total_orders.show()

### Total Order Quantities by Location


In [22]:
# Grouping total order quantities by location
orders_by_location_df = df.groupby('Location')['Order quantities'].sum().reset_index()
orders_by_location_df = orders_by_location_df.sort_values(by='Order quantities', ascending=False)

print("\n--- Total Order Quantities by Location ---")
print(orders_by_location_df)


--- Total Order Quantities by Location ---
    Location  Order quantities
3    Kolkata              1228
1    Chennai              1109
4     Mumbai              1083
0  Bangalore               769
2      Delhi               733


In [23]:
# Bar chart showing total order quantities by location
fig_orders_by_location = px.bar(
    orders_by_location_df,
    x='Location',
    y='Order quantities',
    title='Total Order Quantities by Location',
    labels={'Location': 'Location', 'Order quantities': 'Total Order Quantities'},
    color='Location',
    color_discrete_sequence=px.colors.qualitative.Dark24,
)

fig_orders_by_location.update_layout(
    xaxis_title="Location",
    yaxis_title="Total Order Quantities",
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    bargap=0.1,
)

fig_orders_by_location.show()

## Cost Analysis


### Manufacturing Costs by Product Type


In [24]:
# Calculating manufacturing costs for each product type
manufacturing_costs_by_product = df.groupby('Product type')['Manufacturing costs'].sum().reset_index()
manufacturing_costs_by_product['Manufacturing costs'] = manufacturing_costs_by_product['Manufacturing costs'].round(2)
manufacturing_costs_by_product = manufacturing_costs_by_product.sort_values(by='Manufacturing costs', ascending=False)

print("\n--- Manufacturing Costs by Product Type ---")
print(manufacturing_costs_by_product)


--- Manufacturing Costs by Product Type ---
  Product type  Manufacturing costs
2     skincare              1959.73
1     haircare              1647.57
0    cosmetics              1119.37


In [25]:
# Bar chart for manufacturing costs by product type
fig_manufacturing_costs = px.bar(
    manufacturing_costs_by_product, 
    x='Product type', 
    y='Manufacturing costs', 
    title='Manufacturing Costs by Product Type',
    labels={'Manufacturing costs': 'Manufacturing Costs ($)'},
    color='Product type',
    color_discrete_sequence=px.colors.qualitative.Dark24_r
)

fig_manufacturing_costs.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Manufacturing Costs ($)",
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    bargap=0.1,
)

fig_manufacturing_costs.show()

### Price vs. Manufacturing Costs and Profit Margin


In [26]:
# Aggregate total price and manufacturing costs by product type
price_costs_summary = df.groupby('Product type').agg(
    Price=('Price', 'sum'),
    Manufacturing_costs=('Manufacturing costs', 'sum')
).reset_index()

# Round the price and manufacturing costs to 2 decimal places
price_costs_summary['Price'] = price_costs_summary['Price'].round(2)
price_costs_summary['Manufacturing_costs'] = price_costs_summary['Manufacturing_costs'].round(2)

# Calculate the profit margin as the difference between price and manufacturing costs
price_costs_summary['Profit_margin'] = (price_costs_summary['Price'] - price_costs_summary['Manufacturing_costs']).round(2)

# Sort the summary by product type
price_costs_summary = price_costs_summary.sort_values(by='Product type')

# Display the result
print("\n--- Price, Manufacturing Costs, and Profit Margin by Product Type ---")
print(price_costs_summary)


--- Price, Manufacturing Costs, and Profit Margin by Product Type ---
  Product type    Price  Manufacturing_costs  Profit_margin
0    cosmetics  1491.39              1119.37         372.02
1     haircare  1564.49              1647.57         -83.08
2     skincare  1890.37              1959.73         -69.36


In [27]:
# Grouped bar chart for comparing prices and manufacturing costs.
fig_price_vs_costs = px.bar(
    price_costs_summary,
    x='Product type',
    y=['Price', 'Manufacturing_costs'],
    title='Comparison of Price and Manufacturing Costs by Product Type',
    labels={'value': 'Cost ($)'},
    color_discrete_sequence=['#d62728', '#e377c2'],
    barmode='group'
)

# Adding annotations for profit margin above each price bar.
for i, row in price_costs_summary.iterrows():
    fig_price_vs_costs.add_annotation(
        x=row['Product type'],
        y=row['Price'] + 5,
        text=f"Profit Margin: ${row['Profit_margin']}",
        showarrow=False,
        font=dict(size=10, color='White'),
    )

# Customizing the layout for better aesthetics.
fig_price_vs_costs.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Cost ($)",
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    bargap=0.2,
)

# Displaying the figure.
fig_price_vs_costs.show()

### Overall Profitability by Product Type


In [28]:
# Calculating total revenue and cost to determine overall profitability by product type.
profitability_overall = df.groupby('Product type').agg(
    Revenue=('Revenue generated', 'sum'),
    Cost=('Costs', 'sum')
).reset_index()

# Calculating the profit for each product type by subtracting cost from revenue.
profitability_overall['Profit'] = (profitability_overall['Revenue'] - profitability_overall['Cost']).round(2)

# Sorting the result by product type for better readability.
profitability_overall = profitability_overall.sort_values(by='Product type')

# Displaying the final summary.
print("\n--- Overall Profitability by Product Type ---")
print(profitability_overall)


--- Overall Profitability by Product Type ---
  Product type        Revenue          Cost     Profit
0    cosmetics  161521.265999  13366.397283  148154.87
1     haircare  174455.390605  17328.862865  157126.53
2     skincare  241628.162133  22229.318068  219398.84


In [29]:
# Creating a bar chart to visualize overall profitability by product type.
fig_overall_profitability = px.bar(
    profitability_overall,          # Data source
    x='Product type',               # X-axis: types of products
    y='Profit',                     # Y-axis: profit values
    title='Overall Profitability by Product Type',  # Chart title
    labels={'Profit': 'Profit ($)'},                # Y-axis label
    color='Profit',                                 # Bar color based on profit
    color_continuous_scale=px.colors.diverging.RdYlGn  # Color scale: Red to Green
)

# Updating the layout of the chart for aesthetics.
fig_overall_profitability.update_layout(
    xaxis_title="Product Type",     # Label for X-axis
    yaxis_title="Profit ($)",       # Label for Y-axis
    font=dict(size=14, color='white'),             # Font style and color
    plot_bgcolor='rgba(0, 0, 0, 0)',                # Transparent plot background
    paper_bgcolor='rgba(0, 0, 0, 0)',               # Transparent outer background
    bargap=0.1                                      # Small gap between bars
)

# Displaying the chart.
fig_overall_profitability.show()

## Lead Time and Defect Analysis


### Average Lead Time by Product Type


In [30]:
# Calculate the average lead time for each product type.
average_lead_time_product = df.groupby('Product type')['Lead times'].mean().reset_index()

# Round the average lead time values to two decimal places and store in a new column.
average_lead_time_product['Average Lead Time'] = average_lead_time_product['Lead times'].round(2)

# Sort the data alphabetically by product type.
average_lead_time_product = average_lead_time_product.sort_values(by='Product type')

# Display the results.
print("\n--- Average Lead Time by Product Type ---")
print(average_lead_time_product)


--- Average Lead Time by Product Type ---
  Product type  Lead times  Average Lead Time
0    cosmetics   15.384615              15.38
1     haircare   15.529412              15.53
2     skincare   16.700000              16.70


In [31]:
# Bar chart to visualize the average lead time for each product type.
fig_avg_lead_time = px.bar(
    average_lead_time_product, 
    x='Product type', 
    y='Average Lead Time',
    title='Average Lead Time by Product Type',
    labels={'Average Lead Time': 'Average Lead Time (days)'},
    color='Average Lead Time',
    color_continuous_scale='viridis',
)

# Update layout to improve appearance.
fig_avg_lead_time.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Average Lead Time (days)",
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    bargap=0.1,
)

# Display the bar chart.
fig_avg_lead_time.show()

### Total Availability


In [32]:
# Calculate the total availability from the dataset.
my_total_availability = df['Availability'].sum()

print(f"\nMy total availability: {my_total_availability}")

# Create an indicator chart to display the total availability.
fig_total_availability = go.Figure()
fig_total_availability.add_trace(go.Indicator(
    mode="number",
    value=my_total_availability,
    title={"text": "Total Availability"},
    domain={'x': [0, 1], 'y': [0, 1]}
))

# Customize the chart layout.
fig_total_availability.update_layout(
    font=dict(size=18, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Show the indicator chart.
fig_total_availability.show()


My total availability: 4840


### Defect Rates by Inspection Results


In [33]:
# Calculate the total defect rates grouped by inspection results.
defect_rates_summary = df.groupby('Inspection results')['Defect rates'].sum().reset_index()

# Calculate the overall total defect rate.
total_defect_rate_overall = df['Defect rates'].sum()

# Calculate the percentage each inspection result contributes to the total defect rate.
defect_rates_summary['Percentage of Total Defect Rate'] = \
    (defect_rates_summary['Defect rates'] / total_defect_rate_overall) * 100

# Calculate the average defect rate per inspection result.
avg_defect_rate_per_inspection = df.groupby('Inspection results')['Defect rates'].mean().reset_index()

# Merge the summed and averaged defect rate data into one table.
merged_defect_results = pd.merge(
    defect_rates_summary,
    avg_defect_rate_per_inspection,
    on='Inspection results',
    suffixes=('_sum', '_avg')
)

# Sort the merged results by total defect rate in descending order.
merged_defect_results = merged_defect_results.sort_values(by='Defect rates_sum', ascending=False)

# Display the final merged defect rates summary.
print("\n--- Defect Rates by Inspection Results ---")
print(merged_defect_results)


--- Defect Rates by Inspection Results ---
  Inspection results  Defect rates_sum  Percentage of Total Defect Rate  \
0               Fail         92.494877                        40.618559   
2            Pending         88.322929                        38.786474   
1               Pass         46.897993                        20.594967   

   Defect rates_avg  
0          2.569302  
2          2.154218  
1          2.039043  


In [34]:
# Create a sunburst chart to visualize defect rates by inspection results.
fig_defect_sunburst = px.sunburst(
    merged_defect_results,
    path=['Inspection results'],
    values='Defect rates_sum',
    hover_data=['Percentage of Total Defect Rate', 'Defect rates_avg'],
    title='Defect Rates by Inspection Results (Sunburst Chart)',
    color='Defect rates_sum',
    color_continuous_scale='RdBu'
)

# Update the chart layout with custom font and transparent background.
fig_defect_sunburst.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the chart.
fig_defect_sunburst.show()

### Average Defect Rate by Product Type


In [35]:
# Calculate the average defect rate for each product type.
avg_defect_rate_product = df.groupby('Product type').agg({'Defect rates': 'mean'}).reset_index()

# Round the defect rate to two decimal places.
avg_defect_rate_product['Defect rates'] = avg_defect_rate_product['Defect rates'].round(2)

# Rename the columns for better readability.
avg_defect_rate_product.columns = ['Product Type', 'Average Defect Rate']

# Print the average defect rate for each product type.
print("\n--- Average Defect Rate by Product Type ---")
print(avg_defect_rate_product)


--- Average Defect Rate by Product Type ---
  Product Type  Average Defect Rate
0    cosmetics                 1.92
1     haircare                 2.48
2     skincare                 2.33


In [36]:
# Create a pie chart to display the average defect rate by product type.
fig_avg_defect_pie = px.pie(
    avg_defect_rate_product,
    names='Product Type',
    values='Average Defect Rate',
    title='Average Defect Rate by Product Type',
    color_discrete_sequence=px.colors.qualitative.Pastel  # Use a pastel color palette
)

# Customize the layout of the pie chart.
fig_avg_defect_pie.update_layout(
    font=dict(size=14, color='white'),
    showlegend=True,
    legend_title_text='Product Type',
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent background
    paper_bgcolor='rgba(0, 0, 0, 0)'  # Transparent paper background
)

# Display the pie chart.
fig_avg_defect_pie.show()

## Transportation and Shipping Analysis


### Frequency of Transportation Modes


In [37]:
# Count the number of occurrences for each transportation mode.
transport_mode_counts = df['Transportation modes'].value_counts()

# Display the frequency of each transportation mode.
print("\n--- Frequency of Transportation Modes ---")
print(transport_mode_counts)


--- Frequency of Transportation Modes ---
Transportation modes
Road    29
Rail    28
Air     26
Sea     17
Name: count, dtype: int64


In [38]:
# Create a pie chart to visualize the frequency of transportation modes.
fig_transport_mode_pie = go.Figure()
fig_transport_mode_pie.add_trace(go.Pie(
    labels=transport_mode_counts.index,         # Transportation mode names
    values=transport_mode_counts.values,        # Frequency of each mode
    textinfo='percent+label',                   # Show percentage and label on the chart
    marker_colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],  # Custom colors
    textposition='inside',                      # Position of the text inside the slices
    hole=0.3                                    # Creates a donut-style chart
))

# Update chart layout with a title and transparent background.
fig_transport_mode_pie.update_layout(
    title='Frequency of Transportation Modes',
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the pie chart.
fig_transport_mode_pie.show()

### Relationship Between Transportation Modes, Lead Time, and Costs


In [39]:
# Summarize total lead times and total costs for each transportation mode.
transport_mode_summary = df.groupby('Transportation modes').agg({
    'Lead times': 'sum',   # Sum of lead times per mode
    'Costs': 'sum'         # Sum of costs per mode
}).reset_index()

# Display the summary table
print("\n--- Lead Times and Costs by Transportation Mode ---")
print(transport_mode_summary)


--- Lead Times and Costs by Transportation Mode ---
  Transportation modes  Lead times         Costs
0                  Air         475  14604.527498
1                 Rail         417  15168.931559
2                 Road         497  16048.193639
3                  Sea         207   7102.925520


In [40]:
# Scatter plot to show the relationship between lead time and cost for each transportation mode.
fig_transport_scatter = go.Figure()
fig_transport_scatter.add_trace(go.Scatter(
    x=transport_mode_summary['Lead times'],  # X-axis: lead times
    y=transport_mode_summary['Costs'],       # Y-axis: costs
    mode='markers',                          # Display as points
    marker=dict(color='blue', size=12),      # Marker styling
    text=transport_mode_summary['Transportation modes'],  # Mode names as hover text
    hovertemplate='<b>Transport Mode</b>: %{text}<br><b>Lead Time</b>: %{x}<br><b>Cost</b>: %{y}'  # Tooltip format
))

# Update layout with titles and styling
fig_transport_scatter.update_layout(
    title='Relationship Between Transportation Modes, Lead Time, and Costs',
    xaxis_title='Lead Time',
    yaxis_title='Costs',
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Show the plot
fig_transport_scatter.show()

### Transportation Routes Analysis


In [41]:
# Count the number of times each transportation route appears
route_counts_data = df['Routes'].value_counts()

# Convert the result into a DataFrame for easier viewing and further analysis
route_counts_df = route_counts_data.reset_index()

# Rename the columns to 'Routes' and 'Count'
route_counts_df.columns = ['Routes', 'Count']

# Print the frequency of each transportation route
print("\n--- Frequency of Transportation Routes ---")
print(route_counts_df)


--- Frequency of Transportation Routes ---
    Routes  Count
0  Route A     43
1  Route B     37
2  Route C     20


In [42]:
# Create a bubble chart to visualize the frequency of transportation routes
fig_route_bubble = px.scatter(
    route_counts_df,
    x='Routes',
    y='Count',
    size='Count',
    hover_name='Routes',
    title='Bubble Chart of Transportation Routes with Count',
    labels={'Routes': 'Transportation Routes', 'Count': 'Frequency'},
    size_max=60  # Set the maximum bubble size
)

# Customize the chart's layout for better appearance
fig_route_bubble.update_layout(
    showlegend=False,
    xaxis_title="Transportation Routes",
    yaxis_title="Frequency",
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
    paper_bgcolor='rgba(0, 0, 0, 0)', # Transparent paper background
)

# Display the chart
fig_route_bubble.show()

### Impact of Routes on Lead Times and Costs


In [43]:
# Summarize the total lead times and costs for each transportation route
route_impact_summary = df.groupby('Routes').agg({
    'Lead times': 'sum',
    'Costs': 'sum'
}).reset_index()

# Sort the summary by lead times in descending order
route_impact_summary = route_impact_summary.sort_values(by='Lead times', ascending=False)

# Round the cost values to 2 decimal places
route_impact_summary['Costs'] = route_impact_summary['Costs'].round(2)

# Display the summarized data
print("\n--- Impact of Routes on Lead Times and Costs ---")
print(route_impact_summary)


--- Impact of Routes on Lead Times and Costs ---
    Routes  Lead times     Costs
1  Route B         637  22039.38
0  Route A         632  20875.77
2  Route C         327  10009.42


In [44]:
# Create a parallel categories plot to visualize the relationship between routes, lead times, and costs
fig_route_parallel = px.parallel_categories(
    route_impact_summary,
    dimensions=['Routes', 'Lead times', 'Costs'],
    color='Lead times',
    title='Impact of Routes on Lead Times and Costs',
    color_continuous_scale=px.colors.diverging.Tealrose
)

# Customize the layout of the chart
fig_route_parallel.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the plot
fig_route_parallel.show()

### Manufacturing Costs by Inspection Results

In [45]:
# Summarize total manufacturing costs based on inspection results
manufacturing_costs_by_inspection = df.groupby('Inspection results').agg({'Manufacturing costs': 'sum'}).reset_index()

# Calculate the overall total of manufacturing costs
total_manufacturing_costs = manufacturing_costs_by_inspection['Manufacturing costs'].sum()

# Calculate the percentage contribution of each inspection category to total manufacturing costs
manufacturing_costs_by_inspection['Percentage Contribution'] = (
    manufacturing_costs_by_inspection['Manufacturing costs'] / total_manufacturing_costs * 100
).round(2)

# Round manufacturing costs and percentage contributions to two decimal places
manufacturing_costs_by_inspection['Manufacturing costs'] = manufacturing_costs_by_inspection['Manufacturing costs'].astype(float).round(2)
manufacturing_costs_by_inspection['Percentage Contribution'] = manufacturing_costs_by_inspection['Percentage Contribution'].astype(float).round(2)

# Sort the results in descending order based on manufacturing costs
manufacturing_costs_by_inspection = manufacturing_costs_by_inspection.sort_values(by='Manufacturing costs', ascending=False)

# Print the final summary
print("\n--- Manufacturing Costs by Inspection Results ---")
print(manufacturing_costs_by_inspection)


--- Manufacturing Costs by Inspection Results ---
  Inspection results  Manufacturing costs  Percentage Contribution
0               Fail              1880.30                    39.78
2            Pending              1785.07                    37.77
1               Pass              1061.30                    22.45


In [46]:
# Visualizing manufacturing costs by inspection results using a pie chart.
fig_manufacturing_costs_pie = px.pie(
    manufacturing_costs_by_inspection,
    names='Inspection results',
    values='Manufacturing costs',
    title='Manufacturing Costs by Inspection Results',
    color_discrete_sequence=px.colors.qualitative.Pastel1
)

fig_manufacturing_costs_pie.update_traces(
    hoverinfo='label+value+percent',
    textinfo='value+percent'
)

fig_manufacturing_costs_pie.update_layout(
    font=dict(size=14, color='white'),
    showlegend=True,
    legend_title_text='Inspection Results',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

fig_manufacturing_costs_pie.show()

## Production and Supply Chain Efficiency


### Relationship between Production Volume, Stock Levels, and Order Quantities


In [47]:
# Calculating the total values for key production and inventory metrics.
total_production_volume = df['Production volumes'].sum()
total_stock_level = df['Stock levels'].sum()
total_order_quantity = df['Order quantities'].sum()

print(f"\nTotal Production Volume: {total_production_volume}")
print(f"Total Stock Level: {total_stock_level}")
print(f"Total Order Quantity: {total_order_quantity}")

summary_metrics_df = pd.DataFrame({
    'Metric': ['Production Volume', 'Stock Level', 'Order Quantity'],
    'Total': [total_production_volume, total_stock_level, total_order_quantity]
})

print("\n--- Summary of Production, Stock, and Order Metrics ---")
print(summary_metrics_df)


Total Production Volume: 56784
Total Stock Level: 4777
Total Order Quantity: 4922

--- Summary of Production, Stock, and Order Metrics ---
              Metric  Total
0  Production Volume  56784
1        Stock Level   4777
2     Order Quantity   4922


In [48]:
# Radar chart to visualize the relationship between production volume, stock levels, and order quantities.
fig_production_radar = go.Figure()

fig_production_radar.add_trace(go.Scatterpolar(
    r=[total_production_volume, total_stock_level, total_order_quantity],
    theta=['Production Volume', 'Stock Level', 'Order Quantity'],
    fill='toself',
    name='Metrics',
    line_color='green'
))

fig_production_radar.update_layout(
    title='Relationship between Production Volume, Stock Level, and Order Quantity',
    font=dict(size=14, color='white'),
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(total_production_volume, total_stock_level, total_order_quantity)],
            color='green'
        )
    ),
    showlegend=True,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

fig_production_radar.show()

### Production Volumes by Location

In [49]:
# Summarize total production volumes for each location.
production_by_location = df.groupby('Location').agg({'Production volumes': 'sum'}).reset_index()

# Sort the results in descending order of production volumes.
production_by_location = production_by_location.sort_values(by='Production volumes', ascending=False)

# Display the summarized production data.
print("\n--- Production Volumes by Location ---")
print(production_by_location)


--- Production Volumes by Location ---
    Location  Production volumes
3    Kolkata               15451
4     Mumbai               13160
1    Chennai               11984
2      Delhi                8362
0  Bangalore                7827


In [50]:
# Create a treemap to visualize production volumes by location.
fig_production_treemap = px.treemap(
    production_by_location,
    path=['Location'],
    values='Production volumes',
    color='Production volumes',
    color_continuous_scale='Viridis',
    title='Production Volumes by Location'
)

# Customize the layout and appearance of the treemap.
fig_production_treemap.update_layout(
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the treemap.
fig_production_treemap.show()

### Percentage of Production Volumes Aligned with Market Demands by Location


In [51]:
# Calculate the percentage share of production volumes for each location.
production_percentage_by_location = df.groupby('Location').agg({'Production volumes': 'sum'}).reset_index()

# Calculate the total production across all locations.
total_production_overall = production_percentage_by_location['Production volumes'].sum()

# Compute the percentage for each location.
production_percentage_by_location['Percentage'] = (
    production_percentage_by_location['Production volumes'] / total_production_overall
) * 100

# Sort the results by production volumes in descending order.
production_percentage_by_location = production_percentage_by_location.sort_values(
    by='Production volumes',
    ascending=False
)

# Print the final summary.
print("\n--- Percentage of Production Volumes by Location ---")
print(production_percentage_by_location)


--- Percentage of Production Volumes by Location ---
    Location  Production volumes  Percentage
3    Kolkata               15451   27.210130
4     Mumbai               13160   23.175542
1    Chennai               11984   21.104536
2      Delhi                8362   14.725979
0  Bangalore                7827   13.783812


In [52]:
# Create a pie chart to visualize the percentage of production volumes by location.
fig_production_percentage_pie = px.pie(
    production_percentage_by_location,
    names='Location',
    values='Percentage',
    title='Percentage of Production Volumes Aligned with Market Demands by Location',
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Customize the layout of the chart.
fig_production_percentage_pie.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the chart.
fig_production_percentage_pie.show()

### Revenue Generated by Price Range


In [53]:
# Summarizing total revenue based on product price.
revenue_by_price = df.groupby('Price').agg({'Revenue generated': 'sum'}).reset_index()

print("\n--- Revenue Generated by Price Range ---")
print(revenue_by_price)


--- Revenue Generated by Price Range ---
        Price  Revenue generated
0    1.699976        2828.348746
1    2.397275        6117.324615
2    3.037689        7888.356547
3    3.170011        5709.945296
4    3.526111        4370.916580
..        ...                ...
95  97.446947        3716.493326
96  97.760086        5924.682567
97  98.031830        9435.762609
98  99.113292        5521.205259
99  99.171329        8653.570926

[100 rows x 2 columns]


In [54]:
# Line chart showing revenue across different price ranges.
fig_revenue_by_price = px.line(
    revenue_by_price,
    x='Price',
    y='Revenue generated',
    title='Revenue Generated by Price Range',
    labels={'Revenue generated': 'Total Revenue ($)'},
    markers=True
)

fig_revenue_by_price.update_layout(
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

fig_revenue_by_price.show()

### Average Order Quantities by Shipping Cost Range


In [55]:
# Summarizing the average number of orders based on shipping cost ranges.
orders_by_shipping_cost = df.groupby('Shipping costs').agg({'Order quantities': 'mean'}).reset_index()

print("\n--- Average Order Quantities by Shipping Cost Range ---")
print(orders_by_shipping_cost)


--- Average Order Quantities by Shipping Cost Range ---
    Shipping costs  Order quantities
0         1.013487              54.0
1         1.019488              21.0
2         1.194252              58.0
3         1.311024              59.0
4         1.325274              85.0
..             ...               ...
95        9.705287              27.0
96        9.716575              37.0
97        9.741292               2.0
98        9.898141              24.0
99        9.929816              27.0

[100 rows x 2 columns]


In [56]:
# Bar chart showing the average order quantities for each shipping cost range.
fig_orders_by_shipping_cost = px.bar(
    orders_by_shipping_cost,
    x='Shipping costs',
    y='Order quantities',
    title='Average Order Quantities by Shipping Cost Range',
    labels={'Order quantities': 'Average Order Quantities'},
    color='Shipping costs',
    color_discrete_sequence=px.colors.qualitative.Bold
)

fig_orders_by_shipping_cost.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis={'categoryorder': 'total descending'}
)

fig_orders_by_shipping_cost.show()

### Relationship between Production Volumes and Manufacturing Costs by Location


In [57]:
# Summarizing total production volumes and total manufacturing costs for each location.
production_cost_by_location = df.groupby('Location').agg({
    'Production volumes': 'sum',
    'Manufacturing costs': 'sum'
}).reset_index()

print("\n--- Production Volumes and Manufacturing Costs by Location ---")
print(production_cost_by_location)


--- Production Volumes and Manufacturing Costs by Location ---
    Location  Production volumes  Manufacturing costs
0  Bangalore                7827          1115.909593
1    Chennai               11984          1022.900869
2      Delhi                8362           733.200383
3    Kolkata               15451          1046.578054
4     Mumbai               13160           808.080426


In [58]:
# Scatter plot showing the relationship between production volumes and manufacturing costs by location.
fig_production_cost_scatter = px.scatter(
    production_cost_by_location, 
    x='Production volumes', 
    y='Manufacturing costs', 
    color='Location',
    size='Production volumes',
    hover_name='Location',
    title='Relationship between Production Volumes and Manufacturing Costs by Location',
    labels={
        'Production volumes': 'Production Volumes',
        'Manufacturing costs': 'Manufacturing Costs'
    }, 
    size_max=30
)

fig_production_cost_scatter.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
    xaxis_title='Production Volumes',
    yaxis_title='Manufacturing Costs'
)

fig_production_cost_scatter.show()

### Shipping Costs by Transportation Mode


In [59]:
# Summarizing the total shipping costs for each transportation mode.
shipping_cost_by_mode = df.groupby('Transportation modes')['Shipping costs'].sum().reset_index()

print("\n--- Shipping Costs by Transportation Mode ---")
print(shipping_cost_by_mode)


--- Shipping Costs by Transportation Mode ---
  Transportation modes  Shipping costs
0                  Air      156.463824
1                 Rail      153.134742
2                 Road      160.721346
3                  Sea       84.494995


In [60]:
# Bar chart to visualize shipping costs by each transportation mode.
fig_shipping_cost_bar = px.bar(
    shipping_cost_by_mode, 
    x='Transportation modes', 
    y='Shipping costs', 
    color='Transportation modes',
    hover_name='Transportation modes',
    title='Shipping Costs by Transportation Mode',
    labels={'Shipping costs': 'Shipping Costs ($)'},
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig_shipping_cost_bar.update_layout(
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis_title='Transportation Modes',
    yaxis_title='Shipping Costs ($)'
)

fig_shipping_cost_bar.show()

### Manufacturing Costs vs. Production Volumes


In [61]:
# Summarizing total manufacturing costs for each level of production volume.
manufacturing_cost_vs_production = df.groupby('Production volumes')['Manufacturing costs'].sum().reset_index()

print("\n--- Manufacturing Costs vs. Production Volumes ---")
print(manufacturing_cost_vs_production)


--- Manufacturing Costs vs. Production Volumes ---
    Production volumes  Manufacturing costs
0                  104            56.766476
1                  109            23.126364
2                  152            55.760493
3                  171            75.401612
4                  173            97.829050
..                 ...                  ...
91                 955             4.465278
92                 963            47.957602
93                 964            19.712993
94                 971            30.688019
95                 985            64.323598

[96 rows x 2 columns]


In [62]:
# Create a scatter plot showing the relationship between production volumes and manufacturing costs,
# with a red trendline to indicate the general pattern.
fig_manufacturing_production_scatter = px.scatter(
    manufacturing_cost_vs_production, 
    x='Production volumes', 
    y='Manufacturing costs', 
    trendline='ols',  # Ordinary Least Squares trendline
    title='Manufacturing Costs vs. Production Volumes',
    labels={'Manufacturing costs': 'Manufacturing Costs ($)'},
    hover_name='Production volumes',
    trendline_color_override='red'
)

# Update the layout of the plot for better visual appearance
fig_manufacturing_production_scatter.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis_title='Production Volumes',
    yaxis_title='Manufacturing Costs ($)'
)

# Show the plot
fig_manufacturing_production_scatter.show()

### Distribution of Manufacturing Costs by Supplier


In [63]:
# Summarize the total manufacturing costs for each supplier.
manufacturing_cost_by_supplier = df.groupby('Supplier name')['Manufacturing costs'].sum().reset_index()

# Display the summarized manufacturing costs by supplier.
print("\n--- Manufacturing Costs by Supplier ---")
print(manufacturing_cost_by_supplier)


--- Manufacturing Costs by Supplier ---
  Supplier name  Manufacturing costs
0    Supplier 1          1221.858734
1    Supplier 2           915.695312
2    Supplier 3           654.511820
3    Supplier 4          1128.775084
4    Supplier 5           805.828374


In [64]:
# Create a bar chart to visualize manufacturing costs by supplier.
fig_manufacturing_cost_supplier = px.bar(
    manufacturing_cost_by_supplier,
    x='Supplier name',
    y='Manufacturing costs',
    title='Distribution of Manufacturing Costs by Supplier',
    labels={'Supplier name': 'Supplier Name', 'Manufacturing costs': 'Manufacturing Costs ($)'},
    color='Supplier name',
    color_discrete_sequence=px.colors.qualitative.Set3_r
)

# Customize the layout: font size and color, background transparency, and x-axis order.
fig_manufacturing_cost_supplier.update_layout(
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis={'categoryorder': 'total descending'}
)

# Show the chart.
fig_manufacturing_cost_supplier.show()

### Distribution of Shipping Costs by Shipping Carriers


In [65]:
# Summarize total shipping costs for each shipping carrier.
shipping_cost_by_carrier = df.groupby('Shipping carriers')['Shipping costs'].sum().reset_index()

# Display the summary of shipping costs by carrier.
print("\n--- Shipping Costs by Shipping Carrier ---")
print(shipping_cost_by_carrier)


--- Shipping Costs by Shipping Carrier ---
  Shipping carriers  Shipping costs
0         Carrier A      155.537831
1         Carrier B      236.897620
2         Carrier C      162.379457


In [66]:
# Bar chart to visualize total shipping costs by each shipping carrier.
fig_shipping_cost_carrier = px.bar(
    shipping_cost_by_carrier,
    x='Shipping carriers',
    y='Shipping costs',
    title='Distribution of Shipping Costs by Shipping Carriers',
    labels={
        'Shipping carriers': 'Shipping Carriers',
        'Shipping costs': 'Shipping Costs ($)'
    },
    color='Shipping carriers',
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Customize layout: font color, size, and axis titles.
fig_shipping_cost_carrier.update_layout(
    font=dict(size=14, color='White'),
    xaxis_title=None,
    yaxis_title='Shipping Costs ($)'
)

# Display the chart.
fig_shipping_cost_carrier.show()

### Total Order Quantities by Transportation Mode


In [67]:
# Summarize total order quantities for each transportation mode.
order_quantity_by_mode = df.groupby('Transportation modes')['Order quantities'].sum().reset_index()

# Display the results.
print("\n--- Total Order Quantities by Transportation Mode ---")
print(order_quantity_by_mode)


--- Total Order Quantities by Transportation Mode ---
  Transportation modes  Order quantities
0                  Air              1341
1                 Rail              1342
2                 Road              1386
3                  Sea               853


In [68]:
# Create a Sunburst chart to show total order quantities by transportation mode.
fig_order_quantity_sunburst = px.sunburst(
    order_quantity_by_mode,
    path=['Transportation modes'],
    values='Order quantities',
    title='Total Order Quantities by Transportation Mode',
    color='Order quantities',
    color_continuous_scale=px.colors.sequential.Blues,
    labels={
        'Transportation modes': 'Transportation Mode',
        'Order quantities': 'Total Order Quantities'
    },
)

# Customize chart layout.
fig_order_quantity_sunburst.update_layout(
    font=dict(size=14, color='White'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the chart.
fig_order_quantity_sunburst.show()

### Relationship between Number of Products Sold and Total Shipping Costs


In [69]:
# Calculate the total shipping costs.
df['Total shipping costs'] = df['Number of products sold'] * df['Shipping costs']

# Display a sample of the data including total shipping costs.
print("\n--- Sample Data Showing Total Shipping Costs ---")
print(df[['Number of products sold', 'Shipping costs', 'Total shipping costs']].head())


--- Sample Data Showing Total Shipping Costs ---
   Number of products sold  Shipping costs  Total shipping costs
0                      802        2.956572           2371.170856
1                      736        9.716575           7151.399032
2                        8        8.054479             64.435834
3                       83        1.729569            143.554191
4                      871        3.890548           3388.667235


In [70]:
# Create a scatter plot to show the relationship between products sold and total shipping costs.
fig_products_sold_shipping_cost = px.scatter(
    df,
    x='Number of products sold',
    y='Total shipping costs',
    size='Price',
    color='Customer demographics',
    hover_name='SKU',
    title='Relationship Between Number of Products Sold and Total Shipping Costs',
    labels={
        'Number of products sold': 'Number of Products Sold',
        'Total shipping costs': 'Total Shipping Costs ($)'
    },
    template='plotly_dark'
)

# Update marker appearance for better visibility.
fig_products_sold_shipping_cost.update_traces(
    marker=dict(line=dict(width=1, color='DarkSlateGrey')),
    selector=dict(mode='markers')
)

# Customize the layout with font and background settings.
fig_products_sold_shipping_cost.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Show the scatter plot.
fig_products_sold_shipping_cost.show()

### Relationship between Manufacturing Costs and Revenue Generated


In [71]:
# Create a scatter plot to visualize the relationship between manufacturing costs and revenue generated.
fig_manufacturing_revenue_scatter = px.scatter(
    df, 
    x='Manufacturing costs', 
    y='Revenue generated', 
    size='Price', 
    color='Product type',
    hover_name='SKU',
    title='Relationship between Manufacturing Costs and Revenue Generated',
    labels={'Manufacturing costs': 'Manufacturing Costs ($)'},
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Dark24
)

# Customize the marker appearance with a dark border.
fig_manufacturing_revenue_scatter.update_traces(
    marker=dict(line=dict(width=1, color='DarkSlateGrey')),
    selector=dict(mode='markers')
)

# Update layout with dark theme and readable font settings.
fig_manufacturing_revenue_scatter.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the scatter plot.
fig_manufacturing_revenue_scatter.show()

### Average Lead Times vs. Shipping Times by Transportation Mode


In [72]:
# Summarize the average shipping times and lead times for each transportation mode.
numeric_cols_transport = ['Shipping times', 'Lead times']
transport_time_summary = df.groupby('Transportation modes')[numeric_cols_transport].mean().reset_index()

print("\n--- Average Lead Times vs. Shipping Times by Transportation Mode ---")
print(transport_time_summary)


--- Average Lead Times vs. Shipping Times by Transportation Mode ---
  Transportation modes  Shipping times  Lead times
0                  Air        5.115385   18.269231
1                 Rail        6.571429   14.892857
2                 Road        4.724138   17.137931
3                  Sea        7.117647   12.176471


In [73]:
# Create a line chart to visualize the relationship between average lead times and shipping times by transportation mode.
fig_lead_shipping_time_line = px.line(
    transport_time_summary, 
    x='Shipping times', 
    y='Lead times', 
    color='Transportation modes',
    title='Average Lead Times vs. Shipping Times by Transportation Mode',
    labels={'Shipping times': 'Shipping Times (days)'},
    template='plotly_dark',
    line_shape='spline'  # Smooth curves instead of straight lines
)

# Add both lines and markers to the plot.
fig_lead_shipping_time_line.update_traces(mode='lines+markers')

# Customize the layout: font and transparent background.
fig_lead_shipping_time_line.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

# Display the chart.
fig_lead_shipping_time_line.show()